In [ ]:
SAMPLE_DIR = "../data/raw/iqa"

# Assume the model is already downloaded and placed in the models directory.
# Use one of the following models based on your system architecture.

# MODEL_CHECKPOINT = "../models/fingertip-obj-amd64.pt" # For AMD64
MODEL_CHECKPOINT = "../models/fingertip-obj-unif-arm64.pt"  # For ARM64

In [ ]:
import cv2
from pathlib import Path
from tqdm import tqdm

images, image_titles = [], []
for image_path in tqdm(sorted(list(Path(SAMPLE_DIR).glob("*.jpg")))):
    image = cv2.imread(str(image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    images.append(image)
    image_titles.append(image_path.name)

In [ ]:
import matplotlib.pyplot as plt


def plot_image(image, title, show_axis=False, fig_size=(8, 8)):
    plt.figure(figsize=fig_size)
    plt.imshow(image, cmap="gray" if len(image.shape) == 2 else None)
    plt.title(title)
    plt.axis("on" if show_axis else "off")
    plt.show()


def plot_images(images, titles=None, rows=1, cols=None, sup_title=None, show_axis=False):
    num_images = len(images)

    if titles is None:
        titles = ["Image {}".format(i + 1) for i in range(num_images)]

    if cols is None:
        cols = num_images // rows + (1 if num_images % rows else 0)

    _, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4))
    cmap = "gray" if len(images[0].shape) == 2 else None  # Assume all images have the same shape

    for i, ax in enumerate(axes.flat):
        if i < num_images:
            ax.imshow(images[i], cmap=cmap)
            ax.set_title(titles[i])
            ax.axis("on" if show_axis else "off")
        else:
            # Do not show the remaining subplots
            ax.axis("off")

    if sup_title:
        plt.suptitle(sup_title, fontsize=18)

    plt.tight_layout()
    plt.show()

In [ ]:
plot_images(images, titles=image_titles, cols=4, rows=3, sup_title="Sample Images")

In [ ]:
import pandas as pd
import seaborn as sns


def show_metrics(metrics):
    # Convert metrics to DataFrame
    df = pd.DataFrame(metrics, columns=["Image", "Metrics"])

    # Convert dictionary in Metrics column to separate columns
    df = pd.concat([df["Image"], df["Metrics"].apply(pd.Series)], axis=1)

    # Melt DataFrame to long format for plotting
    df_melted = df.melt(id_vars="Image", var_name="Metric", value_name="Value")

    plt.figure(figsize=(20, 8))  # Increase figure size
    sns.barplot(x="Image", y="Value", hue="Metric", data=df_melted)
    plt.title("Image Quality Metrics")
    plt.xticks(rotation=45, fontsize=8)  # Rotate labels and reduce font size
    plt.show()

In [ ]:
from ultralytics import YOLO

model = YOLO(MODEL_CHECKPOINT)
model.info()
results = model(images, stream=True)

predicted_images = []
predicted_images_titles = []
bbox_coords = []
fingertip_images = []
fingertip_images_titles = []

for result, title in zip(results, image_titles):
    boxes = result.boxes.xyxy.tolist()
    if not boxes:
        continue
    boxes = [int(coord) for coord in boxes[0]]
    bbox_coords.append(boxes)

    original = result.orig_img
    x1, y1, x2, y2 = boxes
    fingertip = original[y1:y2, x1:x2]
    fingertip_images.append(fingertip)
    fingertip_images_titles.append(title)

    predicted = result.plot()
    predicted_images.append(predicted)
    predicted_images_titles.append(title)

In [ ]:
plot_images(
    fingertip_images, titles=fingertip_images_titles, cols=4, rows=2, sup_title="Sample Images"
)

In [ ]:
from mobiofp.background import BackgroundRemoval

remover = BackgroundRemoval()
fingertip_masks = [remover.apply(fingertip) for fingertip in fingertip_images]

In [ ]:
plot_images(fingertip_masks, titles=fingertip_images_titles, cols=4, rows=2)

In [ ]:
fingertip_gray_images = [
    cv2.cvtColor(fingertip, cv2.COLOR_RGB2GRAY) for fingertip in fingertip_images
]
fingertip_gray_images = [
    cv2.normalize(fingertip, None, 0, 255, cv2.NORM_MINMAX) for fingertip in fingertip_gray_images
]
fingertip_nobg_images = [
    cv2.bitwise_and(fingertip, fingertip, mask=mask)
    for fingertip, mask in zip(fingertip_gray_images, fingertip_masks)
]
plot_images(fingertip_nobg_images, titles=fingertip_images_titles, cols=4, rows=2)

In [ ]:
from mobiofp.iqa import compute_quality_metrics
from mobiofp.iqa import gradient_magnitude

fingertip_red_channel = [fingertip[:, :, 1] for fingertip in fingertip_images]
metrics = [
    (Path(title).stem, compute_quality_metrics(image))
    for image, title in zip(fingertip_red_channel, image_titles)
]
gradient_images = [gradient_magnitude(image) for image in fingertip_red_channel]

In [ ]:
plot_images(
    fingertip_red_channel, titles=image_titles, cols=4, rows=2, sup_title="Grayscale Images"
)
plot_images(
    gradient_images, titles=image_titles, cols=4, rows=2, sup_title="Gradient Magnitude Images"
)
show_metrics(metrics)

for title, metric in metrics:
    print(f"{title}: {metric}")

In [ ]:
import numpy as np


def enh1(image, diameter=7, sigma_color=100, sigma_space=100, clip_limit=2.0, tile_grid_size=8):
    enhanced = cv2.bilateralFilter(
        image, d=diameter, sigmaColor=sigma_color, sigmaSpace=sigma_space
    )
    enhanced = cv2.createCLAHE(
        clipLimit=clip_limit, tileGridSize=(tile_grid_size, tile_grid_size)
    ).apply(enhanced)

    return enhanced


def binarize(image, blk_size=11, c=2, inverted=False, method=cv2.ADAPTIVE_THRESH_MEAN_C):
    if inverted:
        return cv2.adaptiveThreshold(image, 255, method, cv2.THRESH_BINARY_INV, blk_size, c)
    return cv2.adaptiveThreshold(image, 255, method, cv2.THRESH_BINARY, blk_size, c)

In [ ]:
# Apply the first enhancement method
enhanced_images1 = [enh1(image) for image in fingertip_gray_images]
grad_enhanced_images1 = [gradient_magnitude(image) for image in enhanced_images1]
binarized_mean1 = [binarize(image) for image in enhanced_images1]
binarized_gaussian1 = [
    binarize(image, method=cv2.ADAPTIVE_THRESH_GAUSSIAN_C) for image in enhanced_images1
]
metrics1 = [
    (Path(title).stem, compute_quality_metrics(image))
    for image, title in zip(enhanced_images1, image_titles)
]

plot_images(
    enhanced_images1, titles=image_titles, cols=4, rows=2, sup_title="Enhanced Images (Method 1)"
)
plot_images(
    grad_enhanced_images1,
    titles=image_titles,
    cols=4,
    rows=2,
    sup_title="Gradient Magnitude Images (Method 1)",
)
show_metrics(metrics1)

for title, metric in metrics1:
    print(f"{title}: {metric}")

plot_images(
    binarized_mean1,
    titles=image_titles,
    cols=4,
    rows=2,
    sup_title="Binarized (Mean) Images (Method 1)",
)
plot_images(
    binarized_gaussian1,
    titles=image_titles,
    cols=4,
    rows=2,
    sup_title="Binarized (Gaussian) Images (Method 1)",
)

In [ ]:
def enh2(image, avg_kernel_size=3, avg_normalize=False, clip_limit=2.0, tile_grid_size=8):
    smoothed = cv2.boxFilter(image, -1, (avg_kernel_size, avg_kernel_size), normalize=avg_normalize)

    # De-blur the image by subtracting the average-filtered image from the original
    deblurred = cv2.absdiff(image, smoothed)
    deblurred = 255 - deblurred

    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
    enhanced = cv2.createCLAHE(
        clipLimit=clip_limit, tileGridSize=(tile_grid_size, tile_grid_size)
    ).apply(deblurred)

    # Sharpen the image using a common 3x3 kernel
    sharpened = cv2.addWeighted(image, 1.5, enhanced, -0.5, 0)

    return sharpened

In [ ]:
# Apply the first enhancement method
enhanced_images2 = [enh2(image) for image in fingertip_gray_images]
grad_enhanced_images2 = [gradient_magnitude(image) for image in enhanced_images2]
binarized_mean2 = [binarize(image) for image in enhanced_images2]
binarized_gaussian2 = [
    binarize(image, method=cv2.ADAPTIVE_THRESH_GAUSSIAN_C) for image in enhanced_images2
]
metrics2 = [
    (Path(title).stem, compute_quality_metrics(image))
    for image, title in zip(enhanced_images2, image_titles)
]

plot_images(
    enhanced_images2, titles=image_titles, cols=4, rows=2, sup_title="Enhanced Images (Method 2)"
)
plot_images(
    grad_enhanced_images2,
    titles=image_titles,
    cols=4,
    rows=2,
    sup_title="Gradient Magnitude Images (Method 2)",
)
show_metrics(metrics2)

for title, metric in metrics2:
    print(f"{title}: {metric}")

plot_images(
    binarized_mean2,
    titles=image_titles,
    cols=4,
    rows=2,
    sup_title="Binarized (Mean) Images (Method 2)",
)
plot_images(
    binarized_gaussian2,
    titles=image_titles,
    cols=4,
    rows=2,
    sup_title="Binarized (Gaussian) Images (Method 2)",
)